In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

PROBLEM = "Can you tell me what level of english I probably have with this knowledge of terminology?"

from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


AttributeError: module 'autogen' has no attribute 'config_list_from_json'

In [9]:
def ask_terminology_expert(message):
    # 1. create an RetrieveAssistantAgent instance named "assistant"
    terminology_assistant = RetrieveAssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant.",
        llm_config={
            "timeout": 600,
            "cache_seed": 42,
            "config_list": config_list,
        },
    )

    # 2. create an UserProxyAgent instance named "user"
    # This agents uses RAG to retrieve information from the document.
    terminology_expert = RetrieveUserProxyAgent(
        name="terminology_expert",
        is_termination_msg=termination_msg,
        system_message="""Assistant who has knowledge of all the terminology that the user knows. 
        Analyzes the terminology and determines the level of the user.
        End the conversation with a summary of the user's level regarding the question/problem given.
        """,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=2,
        retrieve_config={
            "task": "code",
            "docs_path": "../data/memory_longterm.json",
            "chunk_token_size": 1000,
            "model": config_list[0]["model"],
            "client": chromadb.PersistentClient(path="/tmp/chromadb"),
            "collection_name": "user1_terminology",
            "get_or_create": True,
        },
        code_execution_config=False,  # we don't want to execute code in this case.
    )

    # 3. initiate chat between "user" and "assistant"
    terminology_assistant.reset()# reset the assistant. Always reset the assistant before starting a new conversation.
    terminology_expert.initiate_chat(terminology_assistant, problem=PROBLEM)
    return terminology_expert.last_message()["content"]
    ## -------------------------------------------------------------------------------------------------- ##

In [10]:
def ask_background_expert(message):
    # 1. create an RetrieveAssistantAgent instance named "assistant"
    background_assistant = RetrieveAssistantAgent(
        name="assistant",
        system_message="""Assistant who has knowledge of all the background of the user. 
        Analyzes the documents and tries to formulate an answer to the given question.
        End the conversation with a summary of the user's level regarding the question/problem given.
        """,
        llm_config={
            "timeout": 600,
            "cache_seed": 42,
            "config_list": config_list,
        },
    )

    # 2. create an UserProxyAgent instance named "user"
    # This agents uses RAG to retrieve information from the document.
    background_expert = RetrieveUserProxyAgent(
        name="background_expert",
        is_termination_msg=termination_msg,
        system_message="Assistant who has knowledge of the background of a user.",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=2,
        retrieve_config={
            "task": "code",
            "docs_path": "../data/background/CV_2023_DataAndAI.pdf",
            "chunk_token_size": 1000,
            "model": config_list[0]["model"],
            "client": chromadb.PersistentClient(path="/tmp/chromadb"),
            "collection_name": "user1_background",
            "get_or_create": True,
        },
        code_execution_config=False,  # we don't want to execute code in this case.
    )

    # 3. initiate chat between "user" and "assistant"
    background_assistant.reset()# reset the assistant. Always reset the assistant before starting a new conversation.
    background_expert.initiate_chat(background_assistant, problem=PROBLEM)
    return background_expert.last_message()["content"]
    ## -------------------------------------------------------------------------------------------------- ##

In [11]:
assistant_for_student = autogen.AssistantAgent(
    name="assistant_for_student",
    system_message="""You are a helpful assistant. 
    First ask the terminology_expert about what terminology the user knows.
    THEN ask the background_expert about the background of the user.'
    THEN conclude what the current level of the user is.
    THEN reply TERMINATE""",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "temperature": 0,
        "functions": [
            {
                "name": "ask_terminology_expert",
                "description": "ask terminology_expert when you need to know the terminology that the user knows.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": """question to ask terminology_expert. Ensure the question includes enough context.
                            The expert does not know the conversation between you and the user unless you share the conversation
                            with the expert.""",
                        },
                    },
                    "required": ["message"],
                },
            },
            {
                "name": "ask_background_expert",
                "description": "ask background_expert when you need to know the background of a user.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": """question to ask background_expert. Ensure the question includes enough context. 
                            The expert does not know the conversation between you and the user unless you share the conversation with the expert.""",
                        },
                    },
                    "required": ["message"],
                },
            }
        ],
    }
)

student = autogen.UserProxyAgent(
    name="student",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "student"},
    function_map={"ask_terminology_expert": ask_terminology_expert, 
                  "ask_background_expert": ask_background_expert},
)

In [6]:
assistant_for_student.reset()
# the assistant receives a message from the student, which contains the task description
student.initiate_chat(
    assistant_for_student,
    message="""Can you tell me about the level of the user's english?""",
)

student (to assistant_for_student):

Can you tell me about the level of the user's english?

--------------------------------------------------------------------------------
assistant_for_student (to student):

***** Suggested function Call: ask_terminology_expert *****
Arguments: 
{
  "message": "Can you provide information about the English terminology that the user knows?"
}
***********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION ask_terminology_expert...
Trying to create collection.


c:\Users\t-mhendrikx\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Directory ../data/memory_longterm.json does not exist.
Directory ../data/memory_longterm.json does not exist.
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


doc_ids:  [['doc_0']]
Adding doc_id doc_0 to context.
terminology_expert (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Can you tell me what level of english I probably have with this knowledge of terminology?

Context is: {
    "terminology I know": [
        "hello", "goodbye", "please", "thank you",
        "yes", "no", "name", "age",
        "family", "friend", "house", "school",
        "book", "pen", "chair", "table",
        "dog", "cat", "apple", "bread",
        "water", "tea", "coffee", "milk",
   

Number of requested results 60 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 100 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 140 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 180 is greater than number of elements in index 1, updating n_results = 1


doc_ids:  [['doc_0']]
doc_ids:  [['doc_0']]
doc_ids:  [['doc_0']]
doc_ids:  [['doc_0']]
No more context, will terminate.
terminology_expert (to assistant):

TERMINATE

--------------------------------------------------------------------------------
student (to assistant_for_student):

***** Response from calling function "ask_terminology_expert" *****
TERMINATE
*******************************************************************

--------------------------------------------------------------------------------


In [ ]:
import chromadb
from chromadb.config import Settings

import chromadb
chroma_client = chromadb.Client()



client = chromadb.PersistentClient(path="/tmp/chromadb")
collection = client.get_collection(name="groupchat")
client.delete_collection(name="groupchat")
